# Plot results of the synthetic test case.

In [ ]:
%load_ext autoreload
%autoreload 2

import os
import numpy as np
import matplotlib.pyplot as plt
import dask
import pandas as pd
import dask.array as da
import xarray as xr
from climate.utils import load_dataset

from dask.distributed import Client, LocalCluster, wait, progress                             
import diesel as ds                                                             
from diesel.scoring import compute_RE_score, compute_CRPS, compute_energy_score, compute_RMSE 
from diesel.estimation import localize_covariance 

In [ ]:
# base_folder = "/storage/homefs/ct19x463/Dev/Climate/Data/"
base_folder = "/home/cedric/PHD/Dev/Climate/Data/"

# results_folder = "/storage/homefs/ct19x463/Dev/DIESEL/reporting/toy_example/results_paper/synthetic/"
results_folder = "/home/cedric/PHD/Dev/DIESEL/reporting/paleoclimate/results/synthetic/"
plots_folder = "/home/cedric/PHD/Dev/DIESEL/reporting/paleoclimate/results/plots_synthetic/"

In [ ]:
cluster = LocalCluster()
client = Client(cluster)

## Define colors manually.

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns

sns.set()
sns.set_style("white")
plt.rcParams["font.family"] = "serif"
plot_params = {
        'font.size': 32, 'font.style': 'normal',
        'axes.labelsize': 'x-small',
        'axes.titlesize':'x-small',
        'legend.fontsize': 'x-small',
        'xtick.labelsize': 'x-small',
        'ytick.labelsize': 'x-small'
        }
plt.rcParams.update(plot_params)

my_palette = sns.color_palette("twilight_shifted_r", 10)
my_palette_r = reversed(sns.color_palette("twilight_shifted_r", 3))
my_palette[6]

In [ ]:
color_aao, color_seq, color_prior = (0.4981443546207415, 0.13569380302451714, 0.314135190862664), (0.7387914002459927, 0.4205367299231533, 0.34913260148542435), (0.8398783988412087, 0.7603990719977968, 0.7136714781112923)
color_true_cov = my_palette[6]

# Synthetic.

In [ ]:
df_scores_synth = pd.read_pickle(os.path.join(results_folder, "scores.pkl"))

In [ ]:
df_melted_synth = pd.melt(df_scores_synth, value_vars=df_scores_synth.columns, var_name="metric", value_name="loss")
df_melted_synth['experiment'] = df_melted_synth['metric']

df_melted_synth.loc[df_melted_synth['experiment'].str.contains("prior"), 'experiment'] = 'Prior'
df_melted_synth.loc[df_melted_synth['experiment'].str.contains("aao loc"), 'experiment'] = 'All-at-once'
df_melted_synth.loc[df_melted_synth['experiment'].str.contains("seq loc"), 'experiment'] = 'Sequential'
df_melted_synth.loc[df_melted_synth['experiment'].str.contains("truecov"), 'experiment'] = 'True covariance'

df_melted_synth.loc[df_melted_synth['metric'].str.contains("RMSE"), 'metric'] = 'RMSE'
df_melted_synth.loc[df_melted_synth['metric'].str.contains("ES"), 'metric'] = 'ES'
df_melted_synth.loc[df_melted_synth['metric'].str.contains("RE"), 'metric'] = 'RE'

In [ ]:
plt.figure(figsize=(10,6))
ax = sns.boxplot(x="experiment", y="loss",
                 data=df_melted_synth.loc[
                     (df_melted_synth['metric'] == 'RMSE') & (df_melted_synth['experiment'] != 'Prior')
                    ], linewidth=2.5,
                order=['Sequential', 'All-at-once', 'True covariance'],
                palette=[color_seq, color_aao, color_true_cov])
ax.set_ylabel('RMSE')
ax.set_xlabel('')
# ax.set_ylim([0, 1.5])
plt.savefig(os.path.join(plots_folder, 'scores_RMSE'), bbox_inches='tight', dpi=200)

In [ ]:
plt.figure(figsize=(10,6))
ax = sns.boxplot(x="experiment", y="loss",
                 data=df_melted_synth.loc[
                     (df_melted_synth['metric'] == 'ES') & (df_melted_synth['experiment'] != 'Prior')
                      ], linewidth=2.5,
                order=['Sequential', 'All-at-once', 'True covariance'],
                palette=[color_seq, color_aao, color_true_cov])
ax.set_ylabel('Energy Score')
ax.set_xlabel('')
# ax.set_ylim([100, 200])
plt.savefig(os.path.join(plots_folder, 'scores_ES'), bbox_inches='tight', dpi=200)

In [ ]:
plt.figure(figsize=(10,6))
# df_melted_mod_n400 = pd.concat([pd.DataFrame({'metric': ['RE'], 'loss': [np.nan], 'experiment': ['Prior']}), df_melted_n400], axis=0)

ax = sns.boxplot(x="experiment", y="loss",
                 data=df_melted_synth.loc[
                     (df_melted_synth['metric'] == 'RE') & (df_melted_synth['experiment'] != 'Prior')
                 ], linewidth=2.5,
                order=['Sequential', 'All-at-once', 'True covariance'],
                palette=[color_seq, color_aao, color_true_cov])
ax.set_ylabel('RMSE Skill Score')
ax.set_xlabel('')
plt.savefig(os.path.join(plots_folder, 'scores_RE'), bbox_inches='tight', dpi=200)

## Ordering

In [ ]:
df_scores_order = pd.read_pickle(os.path.join(results_folder, "synthetic_ordering/scores.pkl"))             

df_melted_order = pd.melt(df_scores_order, value_vars=df_scores_order.columns, var_name="metric", value_name="loss",)
df_melted_order['experiment'] = df_melted_order['metric']

df_melted_order.loc[df_melted_order['experiment'].str.contains("prior"), 'experiment'] = 'Prior'
df_melted_order.loc[df_melted_order['experiment'].str.contains("aao loc"), 'experiment'] = 'All-at-once'
df_melted_order.loc[df_melted_order['experiment'].str.contains("seq loc"), 'experiment'] = 'Sequential'
df_melted_order.loc[df_melted_order['experiment'].str.contains("truecov"), 'experiment'] = 'True covariance'

df_melted_order.loc[df_melted_order['metric'].str.contains("RMSE"), 'metric'] = 'RMSE'
df_melted_order.loc[df_melted_order['metric'].str.contains("ES"), 'metric'] = 'ES'
df_melted_order.loc[df_melted_order['metric'].str.contains("RE"), 'metric'] = 'RE'

In [ ]:
plt.figure(figsize=(10,6))
ax = sns.boxplot(x="experiment", y="loss",
                 data=df_melted_order.loc[
                     (df_melted_order['metric'] == 'ES') & (df_melted_order['experiment'] != 'Prior')
                      ], linewidth=2.5,
                palette=[color_seq, color_aao])
ax.set_ylabel('Energy Score')
ax.set_xlabel('')
# ax.set_ylim([100, 210])
plt.savefig(os.path.join(plots_folder, 'scores_ES_ordering'), bbox_inches='tight', dpi=200)

In [ ]:
plt.figure(figsize=(10,6))
ax = sns.boxplot(x="experiment", y="loss",
                 data=df_melted_order.loc[
                     (df_melted_order['metric'] == 'RMSE') & (df_melted_order['experiment'] != 'Prior')
                    ], linewidth=2.5,
                palette=[color_seq, color_aao])
ax.set_ylabel('RMSE')
ax.set_xlabel('')
# ax.set_ylim([1, 4])
plt.savefig(os.path.join(plots_folder, 'scores_RMSE_ordering'), bbox_inches='tight', dpi=200)

In [ ]:
plt.figure(figsize=(10,6))
# df_melted_mod_n400 = pd.concat([pd.DataFrame({'metric': ['RE'], 'loss': [np.nan], 'experiment': ['Prior']}), df_melted_n400], axis=0)

ax = sns.boxplot(x="experiment", y="loss",
                 data=df_melted_order[df_melted_order['metric'] == 'RE'], linewidth=2.5,
                palette=[color_seq, color_aao])
ax.set_ylabel('RMSE Skill Score')
ax.set_xlabel('')
plt.savefig(os.path.join(plots_folder, 'scores_RE_ordering'), bbox_inches='tight', dpi=200)

## Plot evolution.

In [ ]:
results_evolution_folder = "/home/cedric/PHD/Dev/DIESEL/reporting/toy_example/results_paper/synthetic_different_noise/"
df_evolution = pd.read_pickle(os.path.join(results_evolution_folder, "scores_merged.pkl"))

In [ ]:
df_evolution['data std'] = 100 * df_evolution['data std']
df_evolution_melted = pd.melt(df_evolution, value_vars=df_evolution.columns, var_name="metric", value_name="loss", id_vars=['data std', 'repetition'])
df_evolution_melted['experiment'] = df_evolution_melted['metric']

df_evolution_melted.loc[df_evolution_melted['experiment'].str.contains("prior"), 'experiment'] = 'Prior'
df_evolution_melted.loc[df_evolution_melted['experiment'].str.contains("aao loc"), 'experiment'] = 'All-at-once'
df_evolution_melted.loc[df_evolution_melted['experiment'].str.contains("seq loc"), 'experiment'] = 'Sequential'
df_evolution_melted.loc[df_evolution_melted['experiment'].str.contains("truecov"), 'experiment'] = 'True covariance'

df_evolution_melted.loc[df_evolution_melted['metric'].str.contains("RMSE"), 'metric'] = 'RMSE'
df_evolution_melted.loc[df_evolution_melted['metric'].str.contains("ES"), 'metric'] = 'ES'
df_evolution_melted.loc[df_evolution_melted['metric'].str.contains("RE"), 'metric'] = 'RE'

In [ ]:
plt.figure(figsize=(10,6))
ax = sns.lineplot(data=df_evolution_melted.loc[(df_evolution_melted['metric'] == 'RMSE') & (df_evolution_melted['experiment'] != 'True covariance')
                                              & (df_evolution_melted['experiment'] != 'Prior')], x="data std", y="loss", hue='experiment',
                 palette=[color_aao, color_seq])
ax.set_ylabel('RMSE')
ax.set_xlim([0, 50])
ax.set_xlabel('Noise std [% of model std]')
leg = plt.legend(fontsize='small', title_fontsize='10')

# change the line width for the legend
for line in leg.get_lines():
    line.set_linewidth(6.0)
    
plt.savefig(os.path.join(plots_folder, 'scores_RMSE_evolution'), bbox_inches='tight', dpi=200)

In [ ]:
plt.figure(figsize=(10,6))
ax = sns.lineplot(data=df_evolution_melted.loc[(df_evolution_melted['metric'] == 'ES') & (df_evolution_melted['experiment'] != 'True covariance')
                                              & (df_evolution_melted['experiment'] != 'Prior')], x="data std", y="loss", hue='experiment',
                 palette=[color_aao, color_seq])
ax.set_ylabel('Energy Score')
ax.set_xlim([0, 50])
ax.set_xlabel('Noise std [% of model std]')
leg = plt.legend(fontsize='small', title_fontsize='10')

# change the line width for the legend
for line in leg.get_lines():
    line.set_linewidth(6.0)
    
plt.savefig(os.path.join(plots_folder, 'scores_ES_evolution'), bbox_inches='tight', dpi=200)

In [ ]:
plt.figure(figsize=(10,6))
ax = sns.lineplot(data=df_evolution_melted.loc[(df_evolution_melted['metric'] == 'RE') & (df_evolution_melted['experiment'] != 'True covariance')], x="data std", y="loss",
                  hue='experiment',
                 palette=[color_aao, color_seq])
ax.set_ylabel('RE Skill Score')
ax.set_xlim([0, 50])
ax.set_xlabel('Noise std [% of model std]')
leg = plt.legend(fontsize='small', title_fontsize='10')

# change the line width for the legend
for line in leg.get_lines():
    line.set_linewidth(6.0)
    
plt.savefig(os.path.join(plots_folder, 'scores_RE_evolution'), bbox_inches='tight', dpi=200)

## Plot spatial situation.

In [ ]:
results_folder_spat = "/home/cedric/PHD/Dev/DIESEL/reporting/toy_example/results_paper/synthetic/"

rep = 0

ground_truth = np.load(os.path.join(results_folder_spat, "ground_truth_{}.npy".format(rep)))
data_inds = np.load(os.path.join(results_folder_spat, "data_inds_{}.npy".format(rep)))

mean = np.load(os.path.join(results_folder_spat, "mean_{}.npy".format(rep)))
ensemble = np.load(os.path.join(results_folder_spat, "ensemble_{}.npy".format(rep)))

mean_updated_aao_loc = np.load(os.path.join(results_folder_spat, "mean_updated_aao_loc_{}.npy".format(rep))).reshape(-1)
ensemble_updated_aao_loc = np.load(os.path.join(results_folder_spat, "ensemble_updated_aao_loc_{}.npy".format(rep)))

mean_updated_aao_truecov = np.load(os.path.join(results_folder_spat, "mean_updated_aao_truecov_{}.npy".format(rep))).reshape(-1)
ensemble_updated_aao_truecov = np.load(os.path.join(results_folder_spat, "ensemble_updated_aao_truecov_{}.npy".format(rep)))

mean_updated_seq_loc = np.load(os.path.join(results_folder_spat, "mean_updated_seq_loc_{}.npy".format(rep))).reshape(-1)
ensemble_updated_seq_loc = np.load(os.path.join(results_folder_spat, "ensemble_updated_seq_loc_{}.npy".format(rep)))

In [ ]:
# Build a square grid with 80^2 elements.
grid = ds.gridding.SquareGrid(n_pts_1d=80)
grid_pts = grid.grid_pts

In [ ]:
cm = 1/2.54  # centimeters in inches

spat_cmap = 'RdBu_r'
# spat_cmap = 'rocket'

# Prior
fig, ax = plt.subplots(1, 1, figsize=(10*cm, 10*cm))
grid.plot_vals(ground_truth, ax, vmin=-3, vmax=3, cmap=spat_cmap, colorbar=True, fig=fig)
plt.savefig(os.path.join(plots_folder, 'ground_truth_synthetic.png'), bbox_inches='tight', dpi=200)

fig, ax = plt.subplots(1, 1, figsize=(10*cm, 10*cm))
grid.plot_vals(mean, ax, vmin=-3, vmax=3, cmap=spat_cmap, colorbar=False, fig=fig)

# Add location of data point.
data_coords = grid.grid_pts[data_inds, :].compute()
ax.scatter(data_coords[:, 0], data_coords[:, 1], s=1, color='black')

plt.savefig(os.path.join(plots_folder, 'mean_synthetic.png'), bbox_inches='tight', dpi=200)
print(compute_RMSE(mean, ground_truth))

fig, ax = plt.subplots(1, 1, figsize=(10*cm, 10*cm))
grid.plot_vals(ensemble[0, :], ax, vmin=-3, vmax=3, cmap=spat_cmap, colorbar=False, fig=fig)
plt.savefig(os.path.join(plots_folder, 'ensemble_0_synthetic.png'), bbox_inches='tight', dpi=200)
print(compute_RMSE(ensemble[0, :], ground_truth))

fig, ax = plt.subplots(1, 1, figsize=(10*cm, 10*cm))
grid.plot_vals(ensemble[1, :], ax, vmin=-3, vmax=3, cmap=spat_cmap, colorbar=False, fig=fig)
plt.savefig(os.path.join(plots_folder, 'ensemble_1_synthetic.png'), bbox_inches='tight', dpi=200)
print(compute_RMSE(ensemble[1, :], ground_truth))

In [ ]:
# All at once.
fig, ax = plt.subplots(1, 1, figsize=(10*cm, 10*cm))
grid.plot_vals(mean_updated_aao_loc, ax, vmin=-3, vmax=3, cmap='RdBu_r', colorbar=False, fig=fig)
plt.savefig(os.path.join(plots_folder, 'mean_updated_aao_loc_synthetic.png'), bbox_inches='tight', dpi=200)
print(compute_RMSE(mean_updated_aao_loc, ground_truth))

fig, ax = plt.subplots(1, 1, figsize=(10*cm, 10*cm))
grid.plot_vals(ensemble_updated_aao_loc[0, :], ax, vmin=-3, vmax=3, cmap='RdBu_r', colorbar=False, fig=fig)
plt.savefig(os.path.join(plots_folder, 'ensemble_updated_aao_loc_0_synthetic.png'), bbox_inches='tight', dpi=200)
print(compute_RMSE(ensemble_updated_aao_loc[0, :], ground_truth))

fig, ax = plt.subplots(1, 1, figsize=(10*cm, 10*cm))
grid.plot_vals(ensemble_updated_aao_loc[1, :], ax, vmin=-3, vmax=3, cmap='RdBu_r', colorbar=False, fig=fig)
plt.savefig(os.path.join(plots_folder, 'ensemble_updated_aao_loc_1_synthetic.png'), bbox_inches='tight', dpi=200)
print(compute_RMSE(ensemble_updated_aao_loc[1, :], ground_truth))

In [ ]:
# Sequential.
fig, ax = plt.subplots(1, 1, figsize=(10*cm, 10*cm))
grid.plot_vals(mean_updated_seq_loc, ax, vmin=-3, vmax=3, cmap='RdBu_r', colorbar=False, fig=fig)
plt.savefig(os.path.join(plots_folder, 'mean_updated_seq_loc_synthetic.png'), bbox_inches='tight', dpi=200)
print(compute_RMSE(mean_updated_seq_loc, ground_truth))

fig, ax = plt.subplots(1, 1, figsize=(10*cm, 10*cm))
grid.plot_vals(ensemble_updated_seq_loc[0, :], ax, vmin=-3, vmax=3, cmap='RdBu_r', colorbar=False, fig=fig)
plt.savefig(os.path.join(plots_folder, 'ensemble_updated_seq_loc_0_synthetic.png'), bbox_inches='tight', dpi=200)
print(compute_RMSE(ensemble_updated_seq_loc[0, :], ground_truth))

fig, ax = plt.subplots(1, 1, figsize=(10*cm, 10*cm))
grid.plot_vals(ensemble_updated_seq_loc[1, :], ax, vmin=-3, vmax=3, cmap='RdBu_r', colorbar=False, fig=fig)
plt.savefig(os.path.join(plots_folder, 'ensemble_updated_seq_loc_1_synthetic.png'), bbox_inches='tight', dpi=200)
print(compute_RMSE(ensemble_updated_seq_loc[1, :], ground_truth))